In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import gseapy as gp
import re
import sys 

import cptac
import cptac.utils as u

/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
en = cptac.Endometrial()
br = cptac.Brca()
cl = cptac.Colon()

In [3]:
gene = 'PIK3CA'

# Filter out hotspot mutations
Hotspot:
E542K
E545K
H1047R


In [4]:
mut = en.get_genotype_all_vars(gene, mutation_hotspot=['E542K', 'E545K', 'H1047R'])
prot = en.get_proteomics()
protfilter = prot.index.str.endswith('.N') #filter out normal samples
prot = prot[~protfilter]

joined = mut.join(prot)
joined['Mutation'].unique()

#select samples containing hotspot mutations
en_hotspot_df = joined[joined.Location.str.contains('E542K') | 
                    joined.Location.str.contains('E545K') |
                    joined.Location.str.contains('H1047R')]
wildtype  = joined.loc[joined.Mutation == "Wildtype_Tumor"]

endo_hotspot = en_hotspot_df.append(wildtype)

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 48 samples for the PIK3CA gene (/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 383)


In [5]:
mut = br.get_genotype_all_vars(gene, mutation_hotspot=['E542K', 'E545K', 'H1047R'])
prot = br.get_proteomics()
protfilter = prot.index.str.endswith('.N') #filter out normal samples
prot = prot[~protfilter]

joined = mut.join(prot)
joined['Mutation'].unique()

#select samples containing hotspot mutations
br_hotspot_df = joined[joined.Location.str.contains('E542K') | 
                    joined.Location.str.contains('E545K') |
                    joined.Location.str.contains('H1047R')]
wildtype  = joined.loc[joined.Mutation == "Wildtype_Tumor"]

brca_hotspot = br_hotspot_df.append(wildtype)
brca_hotspot

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 83 samples for the PIK3CA gene (/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 383)
/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:618: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


,Mutation,Location,Mutation_Status,"(A1BG, NP_570602.2)","(A2M, NP_000005.2)","(A2ML1, NP_653271.2|NP_001269353.1)","(AAAS, NP_056480.1|NP_001166937.1)","(AACS, NP_076417.2|NP_001306769.1|NP_001306768.1)","(AADAT, NP_057312.1|NP_001273611.1)","(AAED1, NP_714542.1)",...,"(ZSCAN31, NP_665916.1|NP_001230172.1|NP_116078.4|NP_001166148.1|NP_001229731.1|NP_001012458.1)","(ZSWIM8, NP_055852.2|NP_001229416.1|NP_001229417.1)","(ZW10, NP_004715.1)","(ZWILCH, NP_060445.3|NP_001274750.1)","(ZWINT, NP_008988.2|NP_001005413.1)","(ZXDC, NP_079388.3|NP_001035743.1)","(ZYG11B, NP_078922.1)","(ZYX, NP_001010972.1)","(ZZEF1, NP_055928.3)","(ZZZ3, NP_056349.1|NP_001295166.1)"
Patient_ID,,,,,,,,,,,,,,,,,,,,,
CPT001846,Missense_Mutation,p.E542K,Multiple_mutation,1.3964,1.3302,-5.0948,0.7674,-1.6845,NaN,2.1022,...,-0.7592,0.4711,0.6018,0.2062,-0.2137,-2.1219,0.0860,2.5814,-0.2852,-0.1074
X11BR004,Missense_Mutation,p.H1047R,Multiple_mutation,1.3627,1.8899,-0.5023,-0.6201,0.8102,NaN,-1.1417,...,1.8955,0.3530,0.8214,1.2056,-0.6622,-0.5864,-0.0059,1.3178,0.4372,-1.0408
X11BR013,Missense_Mutation,p.H1047R,Single_mutation,0.7803,-0.6980,-3.5729,0.7011,-1.8474,-2.7737,-0.0788,...,NaN,1.0010,-0.6068,0.8283,-1.0147,1.4330,0.2379,-1.0891,-0.2156,-0.3116
X11BR014,Missense_Mutation,p.E545K,Multiple_mutation,0.9126,-0.2126,-8.9169,-0.6406,-1.9189,-2.4744,-2.0662,...,1.2953,-0.1190,0.1474,-1.5504,-3.3162,-0.5357,1.4172,1.8508,0.2182,-0.5697
X11BR022,Missense_Mutation,p.H1047R,Single_mutation,1.9095,1.4993,-4.9660,-0.5247,-0.2128,-3.1250,-0.2501,...,NaN,-0.3823,-0.0501,0.1364,-0.6264,-0.8129,-1.1180,-0.5281,-0.6841,0.0143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X15BR003,Wildtype_Tumor,No_mutation,Wildtype_Tumor,2.4710,1.0447,1.6000,-0.2389,-0.2619,NaN,1.2958,...,-0.3656,-1.0569,0.8718,-0.4324,-0.7942,-2.4072,1.2313,0.6114,-0.6675,-0.8818
X18BR004,Wildtype_Tumor,No_mutation,Wildtype_Tumor,-0.4652,-0.2157,-3.6000,-0.3496,-0.9725,NaN,NaN,...,-2.0780,0.9906,0.3147,0.0305,-0.2289,NaN,0.2189,-0.3281,0.4254,-0.1496
X18BR006,Wildtype_Tumor,No_mutation,Wildtype_Tumor,0.0999,0.7852,-8.1266,-0.5701,-2.1021,NaN,-0.5732,...,-0.3722,0.0816,0.0086,0.3131,-0.2260,-0.6128,0.7913,0.8553,-0.0371,-0.3874


# Run T-test and difference of Median for each cancer type

# Endo get difference in median and t-test

In [6]:
prot_and_mutations = endo_hotspot.drop(columns = ["Mutation_Status", "Location"])
prot_and_mutations

# Keep two values to compare
compare = ['Wildtype_Tumor','Missense_Mutation']
get = prot_and_mutations['Mutation'].isin(compare)
missense_wt = prot_and_mutations[get]
missense_wt['Mutation'].value_counts()


# Step 2 get the difference in medians
missense = missense_wt[missense_wt.Mutation == "Missense_Mutation"] #get all missense_mutations
wt = missense_wt[missense_wt.Mutation == "Wildtype_Tumor"] #getn all wildtype
missense_med = missense.median() #get the medain of the missense. This will give the median expression among all samples, for each gene
wt_med = wt.median() #get the median of the wildtype
wt_med

endo_d = {}

for prot in prot_and_mutations:
    if prot == "Mutation": continue
    dif_endo = wt_med[prot] - missense_med[prot]
    endo_d[prot] = dif_endo
#     print(wt_med[prot], '-', missense_med[prot], '=',dif_endo)

median_diff = pd.DataFrame.from_dict(endo_d, orient='index', columns=['Difference_In_Median'])
median_diff

#step 3 do a t test for every gene and report the p-value
cols = list(missense_wt.columns[1:])
t_test = u.wrap_ttest(missense_wt, 'Mutation', cols, return_all = True)
t_test = t_test.set_index('Comparison')

# Step 4 join median and p value together
endo_df = median_diff.join(t_test)
endo_df

/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3367: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/statsmodels/stats/multitest.py:153: RuntimeWarning: invalid value encountered in less_equal
  reject = pvals <= alphacBonf
/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/statsmodels/stats/multitest.py:257: RuntimeWarning: invalid value encountered in greater
  pvals_corrected[pvals_corrected>1] = 1


,Difference_In_Median,P_Value
A1BG,-0.055500,0.434646
A2M,-0.038000,0.714215
A2ML1,-1.105850,0.037044
A4GALT,-0.111075,0.837021
AAAS,-0.156100,0.664545
...,...,...
ZXDC,-0.295450,0.651730
ZYG11B,0.109900,0.102762
ZYX,0.419000,0.079347
ZZEF1,0.130900,0.486940


# Brca get difference in median and t-test

In [7]:
br = cptac.Brca()

In [8]:
#separate proteomics into genes with duplicate database ID and genes with only one databaseID
proteomics = br.get_proteomics()
fil = proteomics.columns.get_level_values('Name').duplicated(False)
duplicates = proteomics[proteomics.columns[fil]]
no_duplicates = proteomics[proteomics.columns[~fil]]

In [9]:
#FOR NO DUPLICATES
#get the pval and min for no duplicates
no_duplicates = br.reduce_multiindex(no_duplicates, flatten=True)

#  Step 1 - Create dataframe in order to do comparisons with wrap_ttest
mut_type = br.get_genotype_all_vars(gene)
# mut_type = mut_type.drop(columns = ["Mutation_Status", "Location"])

joined = mut_type.join(no_duplicates)
joined

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 83 samples for the PIK3CA gene (/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 383)


Name,Mutation,Location,Mutation_Status,A1BG_NP_570602.2,A2M_NP_000005.2,A2ML1_NP_653271.2|NP_001269353.1,AAAS_NP_056480.1|NP_001166937.1,AACS_NP_076417.2|NP_001306769.1|NP_001306768.1,AADAT_NP_057312.1|NP_001273611.1,AAED1_NP_714542.1,...,ZSCAN31_NP_665916.1|NP_001230172.1|NP_116078.4|NP_001166148.1|NP_001229731.1|NP_001012458.1,ZSWIM8_NP_055852.2|NP_001229416.1|NP_001229417.1,ZW10_NP_004715.1,ZWILCH_NP_060445.3|NP_001274750.1,ZWINT_NP_008988.2|NP_001005413.1,ZXDC_NP_079388.3|NP_001035743.1,ZYG11B_NP_078922.1,ZYX_NP_001010972.1,ZZEF1_NP_055928.3,ZZZ3_NP_056349.1|NP_001295166.1
Patient_ID,,,,,,,,,,,,,,,,,,,,,
CPT000814,Amplification,Amplification,Single_mutation,-0.6712,-0.2075,2.7959,1.3969,-1.0899,NaN,1.6708,...,-5.2868,-0.6536,0.3384,2.1169,1.3910,-2.1230,0.9136,-0.8082,-1.4793,0.9136
CPT001846,Missense_Mutation,p.E542K,Multiple_mutation,1.3964,1.3302,-5.0948,0.7674,-1.6845,NaN,2.1022,...,-0.7592,0.4711,0.6018,0.2062,-0.2137,-2.1219,0.0860,2.5814,-0.2852,-0.1074
X01BR001,Wildtype_Tumor,No_mutation,Wildtype_Tumor,2.0219,1.6269,-3.2943,0.3352,-1.0739,1.2255,0.2754,...,NaN,0.2306,-0.3010,0.3395,-0.5316,NaN,0.4996,0.7622,-1.5607,0.0256
X01BR008,Wildtype_Tumor,No_mutation,Wildtype_Tumor,-0.5290,0.3267,1.4342,0.4938,-2.8676,NaN,NaN,...,-2.1789,0.2695,0.1506,1.0498,0.7546,1.7889,-0.2499,-0.2590,-0.1263,0.3725
X01BR009,Amplification,Amplification,Single_mutation,1.2556,3.4489,2.8043,-0.2956,-1.7261,NaN,NaN,...,-2.3990,-0.2596,0.1898,-0.5010,-0.4189,0.3080,0.5057,0.2181,-0.2288,-0.2750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X21BR001,Amplification,Amplification,Single_mutation,-0.6610,-0.6402,-4.8578,1.2319,-1.6491,NaN,NaN,...,-0.2528,0.5090,0.0306,0.4908,-0.5570,2.3864,0.3764,-0.6974,1.3541,1.1123
X21BR002,Missense_Mutation,p.H1047R,Multiple_mutation,-1.3735,0.4227,-4.9553,0.6327,-3.1434,NaN,NaN,...,-3.3351,0.1548,1.0792,-0.6619,-1.4444,-0.3704,0.4909,0.3938,0.2992,-0.3494
X21BR010,Missense_Mutation,p.H1047R,Multiple_mutation,1.1583,0.3329,-5.7358,-0.1658,-2.0413,-1.2433,0.9090,...,-0.7054,0.2752,0.8850,-2.6704,-0.9444,-1.9717,0.0650,0.6300,-0.0686,0.1798


In [10]:
#select samples containing hotspot mutations
br_hotspot_df = joined[joined.Location.str.contains('E542K') | 
                    joined.Location.str.contains('E545K') |
                    joined.Location.str.contains('H1047R')]
wildtype  = joined.loc[joined.Mutation == "Wildtype_Tumor"]

braca_hotspot = br_hotspot_df.append(wildtype)
braca_hotspot = braca_hotspot.drop(columns = ["Mutation_Status", "Location"])

braca_hotspot

Name,Mutation,A1BG_NP_570602.2,A2M_NP_000005.2,A2ML1_NP_653271.2|NP_001269353.1,AAAS_NP_056480.1|NP_001166937.1,AACS_NP_076417.2|NP_001306769.1|NP_001306768.1,AADAT_NP_057312.1|NP_001273611.1,AAED1_NP_714542.1,AAGAB_NP_078942.3|NP_001258815.1,AAK1_NP_055726.3,...,ZSCAN31_NP_665916.1|NP_001230172.1|NP_116078.4|NP_001166148.1|NP_001229731.1|NP_001012458.1,ZSWIM8_NP_055852.2|NP_001229416.1|NP_001229417.1,ZW10_NP_004715.1,ZWILCH_NP_060445.3|NP_001274750.1,ZWINT_NP_008988.2|NP_001005413.1,ZXDC_NP_079388.3|NP_001035743.1,ZYG11B_NP_078922.1,ZYX_NP_001010972.1,ZZEF1_NP_055928.3,ZZZ3_NP_056349.1|NP_001295166.1
Patient_ID,,,,,,,,,,,,,,,,,,,,,
CPT001846,Missense_Mutation,1.3964,1.3302,-5.0948,0.7674,-1.6845,NaN,2.1022,-0.5814,0.2916,...,-0.7592,0.4711,0.6018,0.2062,-0.2137,-2.1219,0.0860,2.5814,-0.2852,-0.1074
X11BR004,Missense_Mutation,1.3627,1.8899,-0.5023,-0.6201,0.8102,NaN,-1.1417,-0.1994,0.3278,...,1.8955,0.3530,0.8214,1.2056,-0.6622,-0.5864,-0.0059,1.3178,0.4372,-1.0408
X11BR013,Missense_Mutation,0.7803,-0.6980,-3.5729,0.7011,-1.8474,-2.7737,-0.0788,0.2499,-1.4275,...,NaN,1.0010,-0.6068,0.8283,-1.0147,1.4330,0.2379,-1.0891,-0.2156,-0.3116
X11BR014,Missense_Mutation,0.9126,-0.2126,-8.9169,-0.6406,-1.9189,-2.4744,-2.0662,-0.2834,0.4592,...,1.2953,-0.1190,0.1474,-1.5504,-3.3162,-0.5357,1.4172,1.8508,0.2182,-0.5697
X11BR022,Missense_Mutation,1.9095,1.4993,-4.9660,-0.5247,-0.2128,-3.1250,-0.2501,1.9603,-0.1348,...,NaN,-0.3823,-0.0501,0.1364,-0.6264,-0.8129,-1.1180,-0.5281,-0.6841,0.0143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X15BR003,Wildtype_Tumor,2.4710,1.0447,1.6000,-0.2389,-0.2619,NaN,1.2958,0.1298,0.3695,...,-0.3656,-1.0569,0.8718,-0.4324,-0.7942,-2.4072,1.2313,0.6114,-0.6675,-0.8818
X18BR004,Wildtype_Tumor,-0.4652,-0.2157,-3.6000,-0.3496,-0.9725,NaN,NaN,0.2205,-0.7924,...,-2.0780,0.9906,0.3147,0.0305,-0.2289,NaN,0.2189,-0.3281,0.4254,-0.1496
X18BR006,Wildtype_Tumor,0.0999,0.7852,-8.1266,-0.5701,-2.1021,NaN,-0.5732,0.4928,-0.2838,...,-0.3722,0.0816,0.0086,0.3131,-0.2260,-0.6128,0.7913,0.8553,-0.0371,-0.3874


In [11]:
# Keep two values to compare
compare = ['Wildtype_Tumor','Missense_Mutation']
get = braca_hotspot['Mutation'].isin(compare)
missense_wt = braca_hotspot[get]
missense_wt['Mutation'].value_counts()

# Step 2 get the difference in medians
missense = missense_wt[missense_wt.Mutation == "Missense_Mutation"] #get all missense_mutations
wt = missense_wt[missense_wt.Mutation == "Wildtype_Tumor"] #getn all wildtype
missense_med = missense.median() #get the medain of the missense. This will give the median expression among all samples, for each gene
wt_med = wt.median() #get the median of the wildtype


no_dup_d = {}

for prot in no_duplicates:
    dif_brca = wt_med[prot] - missense_med[prot]
    no_dup_d[prot] = dif_brca
#     print(wt_med[prot], '-', missense_med[prot], '=',dif_endo)

median_diff = pd.DataFrame.from_dict(no_dup_d, orient='index', columns=['Difference_In_Median'])

#step 3 do a t test for every gene and report the p-value
cols = list(missense_wt.columns[1:])
t_test = u.wrap_ttest(missense_wt, 'Mutation', cols, return_all = True)
t_test = t_test.set_index('Comparison')

# Step 4 join median and p value together
no_dup_df = median_diff.join(t_test)
no_dup_df

/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3367: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/statsmodels/stats/multitest.py:153: RuntimeWarning: invalid value encountered in less_equal
  reject = pvals <= alphacBonf
/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/statsmodels/stats/multitest.py:257: RuntimeWarning: invalid value encountered in greater
  pvals_corrected[pvals_corrected>1] = 1


,Difference_In_Median,P_Value
A1BG_NP_570602.2,-0.63510,0.830859
A2M_NP_000005.2,-0.05110,0.999049
A2ML1_NP_653271.2|NP_001269353.1,1.26120,0.165194
AAAS_NP_056480.1|NP_001166937.1,-0.13550,0.201707
AACS_NP_076417.2|NP_001306769.1|NP_001306768.1,0.01775,0.652935
...,...,...
ZXDC_NP_079388.3|NP_001035743.1,0.32175,0.925221
ZYG11B_NP_078922.1,-0.05100,0.846339
ZYX_NP_001010972.1,-0.49420,0.373013
ZZEF1_NP_055928.3,-0.09520,0.300908


In [12]:
#DEALING WITH DUPLICATES
#get the pval and min for duplicates

duplicates = br.reduce_multiindex(duplicates, flatten=True)

# We need to figure which of databaseID we want to use, for each protein
proteomics = br.get_proteomics()
protfilter = proteomics.index.str.endswith('.N') #filter out normal samples
fil = proteomics.columns.get_level_values('Name').duplicated(False)
duplicates = proteomics[proteomics.columns[fil]]
duplicate_gene_names = list(set(duplicates.columns.get_level_values('Name'))) #get a list of genes names that are duplicated
duplicates = br.reduce_multiindex(duplicates, flatten=True)
duplicates

#  Step 1 - Create dataframe in order to do comparisons with wrap_ttest

mut_type = br.get_genotype_all_vars(gene)


joined = mut_type.join(duplicates)

#select samples containing hotspot mutations
br_hotspot_df = joined[joined.Location.str.contains('E542K') | 
                    joined.Location.str.contains('E545K') |
                    joined.Location.str.contains('H1047R')]
wildtype  = joined.loc[joined.Mutation == "Wildtype_Tumor"]

brace_hotspot = br_hotspot_df.append(wildtype)
brace_hotspot = brace_hotspot.drop(columns = ["Mutation_Status", "Location"])

# Keep two values to compare
compare = ['Wildtype_Tumor','Missense_Mutation']
get = brace_hotspot['Mutation'].isin(compare)
missense_wt = brace_hotspot[get]
missense_wt['Mutation'].value_counts()

# Step 2 get the difference in medians
missense = missense_wt[missense_wt.Mutation == "Missense_Mutation"] #get all missense_mutations
wt = missense_wt[missense_wt.Mutation == "Wildtype_Tumor"] #getn all wildtype
missense_med = missense.median() #get the medain of the missense. This will give the median expression among all samples, for each gene
wt_med = wt.median() #get the median of the wildtype


duplicates_d = {}

for prot in duplicates:
    dif_brca = wt_med[prot] - missense_med[prot]
    duplicates_d[prot] = dif_brca
#     print(wt_med[prot], '-', missense_med[prot], '=',dif_endo)

median_diff = pd.DataFrame.from_dict(duplicates_d, orient='index', columns=['Difference_In_Median'])

#step 3 do a t test for every gene and report the p-value
cols = list(missense_wt.columns[1:])
t_test = u.wrap_ttest(missense_wt, 'Mutation', cols, return_all = True)
t_test = t_test.set_index('Comparison')

# Step 4 join median and p value together
dup_df = median_diff.join(t_test)
dup_df

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 83 samples for the PIK3CA gene (/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 383)


,Difference_In_Median,P_Value
ABCB7_NP_001258628.1|NP_001258626.1,0.29320,0.238399
ABCB7_NP_004290.2|NP_001258625.1|NP_001258627.1,0.25730,0.537629
ABLIM3_NP_001287947.1|NP_001287944.1,-0.18300,0.781017
ABLIM3_NP_001287956.1,0.48620,0.166418
ACIN1_NP_001158286.1,-0.15200,0.783389
...,...,...
YBX3_NP_003642.3,0.10470,0.892280
ZAK_NP_057737.2,-0.12710,0.650638
ZAK_NP_598407.1,-0.45065,0.765156
ZNF185_NP_001171579.1|NP_001171584.1,-0.46810,0.282673


In [13]:
#for each duplicated gene name select the one with the min p val and append to no_duplicat df
for name in duplicate_gene_names:
    gene_duplicate = dup_df[dup_df.index.str.contains(name)]
    #select the one with the min p value for t test
    min_row = gene_duplicate[gene_duplicate.P_Value == gene_duplicate.P_Value.min()]
    #append the selected version onto no_duplicate dataframe
    no_dup_df=no_dup_df.append(min_row)
no_dup_df=no_dup_df.sort_index()
no_dup_df

,Difference_In_Median,P_Value
A1BG_NP_570602.2,-0.63510,0.830859
A2ML1_NP_653271.2|NP_001269353.1,1.26120,0.165194
A2M_NP_000005.2,-0.05110,0.999049
AAAS_NP_056480.1|NP_001166937.1,-0.13550,0.201707
AACS_NP_076417.2|NP_001306769.1|NP_001306768.1,0.01775,0.652935
...,...,...
ZXDC_NP_079388.3|NP_001035743.1,0.32175,0.925221
ZYG11B_NP_078922.1,-0.05100,0.846339
ZYX_NP_001010972.1,-0.49420,0.373013
ZZEF1_NP_055928.3,-0.09520,0.300908


In [14]:
#reformat the table by getting rid of databaseIDs so we can join it to Endo and BRca
text='A2ML1_NP_653271.2|NP_001269353.1'
sep = '_'
text.split(sep)[0]

'A2ML1'

In [15]:
#reformat the table by getting rid of databaseIDs so we can join it to Endo and BRca
split = no_dup_df.index.str.split(sep, 1).str[0]
no_dup_df['new_index']=split
brca_df = no_dup_df.set_index("new_index")
# split["A"].str[0]

In [16]:
brca_df 

,Difference_In_Median,P_Value
new_index,,
A1BG,-0.63510,0.830859
A2ML1,1.26120,0.165194
A2M,-0.05110,0.999049
AAAS,-0.13550,0.201707
AACS,0.01775,0.652935
...,...,...
ZXDC,0.32175,0.925221
ZYG11B,-0.05100,0.846339
ZYX,-0.49420,0.373013


# Colon get difference in median and t-test

In [17]:
cl = cptac.Colon()

In [18]:
#  Step 1 - Create dataframe in order to do comparisons with wrap_ttest
proteomics = cl.get_proteomics()
protfilter = proteomics.index.str.endswith('.N')
proteomics = proteomics[~protfilter]

prot_list = list(proteomics.columns)

mut_type = cl.get_genotype_all_vars(gene)


joined = mut_type.join(proteomics)

#select samples containing hotspot mutations
cl_hotspot_df = joined[joined.Location.str.contains('E542K') | 
                    joined.Location.str.contains('E545K') |
                    joined.Location.str.contains('H1047R')]
wildtype  = joined.loc[joined.Mutation == "Wildtype_Tumor"]

colon_hotspot = cl_hotspot_df.append(wildtype)
colon_hotspot = colon_hotspot.drop(columns = ["Mutation_Status", "Location"])


# Keep two values to compare
compare = ['Wildtype_Tumor','nonsynonymous SNV']
get = colon_hotspot['Mutation'].isin(compare)
missense_wt = colon_hotspot[get]
missense_wt['Mutation'].value_counts()

# Step 2 get the difference in medians
missense = missense_wt[missense_wt.Mutation == "nonsynonymous SNV"] #get all missense_mutations
wt = missense_wt[missense_wt.Mutation == "Wildtype_Tumor"] #getn all wildtype
missense_med = missense.median() #get the medain of the missense. This will give the median expression among all samples, for each gene
wt_med = wt.median() #get the median of the wildtype


colon_d = {}

for prot in proteomics:
    dif_colon = wt_med[prot] - missense_med[prot]
    colon_d[prot] = dif_colon
#     print(wt_med[prot], '-', missense_med[prot], '=',dif_endo)

median_diff = pd.DataFrame.from_dict(colon_d, orient='index', columns=['Difference_In_Median'])

#step 3 do a t test for every gene and report the p-value
cols = list(missense_wt.columns[1:])
t_test = u.wrap_ttest(missense_wt, 'Mutation', cols, return_all = True)
t_test = t_test.set_index('Comparison')

# Step 4 join median and p value together
colon_df = median_diff.join(t_test)
colon_df

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 82 samples for the PIK3CA gene (/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 383)
cptac warning: CNV data was not found for the following samples, so CNV data columns were filled with NaN for these samples: 05CO045 (/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 383)
/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3367: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/statsmodels/stats/multitest.py:153: RuntimeWarning: invalid value encountered in less_equal
  rejec

,Difference_In_Median,P_Value
A1BG,0.61600,0.263957
A1CF,0.03225,0.913020
A2M,0.26800,0.265891
AAAS,0.26315,0.070194
AACS,0.00850,0.113686
...,...,...
ZW10,0.05110,0.446415
ZWILCH,0.15380,0.231705
ZWINT,NaN,NaN
ZYX,-0.06050,0.985312


# Combine Endo, Brca, and Colon into one table

In [19]:
brca_df=brca_df.rename_axis(None)

In [20]:
brca_endo = brca_df.join(endo_df, lsuffix='_Brca', rsuffix='_Endo')
brca_endo

,Difference_In_Median_Brca,P_Value_Brca,Difference_In_Median_Endo,P_Value_Endo
A1BG,-0.63510,0.830859,-0.055500,0.434646
A2M,-0.05110,0.999049,-0.038000,0.714215
A2ML1,1.26120,0.165194,-1.105850,0.037044
AAAS,-0.13550,0.201707,-0.156100,0.664545
AACS,0.01775,0.652935,-0.021500,0.732714
...,...,...,...,...
ZXDC,0.32175,0.925221,-0.295450,0.651730
ZYG11B,-0.05100,0.846339,0.109900,0.102762
ZYX,-0.49420,0.373013,0.419000,0.079347
ZZEF1,-0.09520,0.300908,0.130900,0.486940


In [21]:
final_table = brca_endo.join(colon_df, rsuffix='_colon')
final_table

,Difference_In_Median_Brca,P_Value_Brca,Difference_In_Median_Endo,P_Value_Endo,Difference_In_Median,P_Value
A1BG,-0.63510,0.830859,-0.055500,0.434646,0.61600,0.263957
A2M,-0.05110,0.999049,-0.038000,0.714215,0.26800,0.265891
A2ML1,1.26120,0.165194,-1.105850,0.037044,NaN,NaN
AAAS,-0.13550,0.201707,-0.156100,0.664545,0.26315,0.070194
AACS,0.01775,0.652935,-0.021500,0.732714,0.00850,0.113686
...,...,...,...,...,...,...
ZXDC,0.32175,0.925221,-0.295450,0.651730,NaN,NaN
ZYG11B,-0.05100,0.846339,0.109900,0.102762,NaN,NaN
ZYX,-0.49420,0.373013,0.419000,0.079347,-0.06050,0.985312
ZZEF1,-0.09520,0.300908,0.130900,0.486940,-0.12420,0.535679


In [22]:
#only keep significant pvals
significant = final_table.loc[(final_table['P_Value_Brca'] <= .05) |
                (final_table['P_Value_Endo'] <= .05) |
                (final_table['P_Value'] <= .05)]
significant

,Difference_In_Median_Brca,P_Value_Brca,Difference_In_Median_Endo,P_Value_Endo,Difference_In_Median,P_Value
A2ML1,1.26120,0.165194,-1.10585,0.037044,NaN,NaN
AADAT,1.33200,0.027477,0.18535,0.690613,NaN,NaN
AAGAB,0.09455,0.909622,-0.24350,0.139632,-0.1390,0.039189
AASDHPPT,-0.51445,0.003620,0.10590,0.428884,0.0628,0.973515
AATF,0.16905,0.042271,0.18170,0.915482,-0.0250,0.568821
...,...,...,...,...,...,...
ZNF654,-0.49290,0.048872,0.11860,0.294513,NaN,NaN
ZNF768,0.35120,0.069655,-0.19580,0.661851,-0.3755,0.030999
ZNHIT3,0.01085,0.990265,-0.13895,0.029161,NaN,NaN
ZNRD1,0.18620,0.423039,-0.30475,0.017492,NaN,NaN


In [34]:
significant

,Difference_In_Median_Brca,P_Value_Brca,Difference_In_Median_Endo,P_Value_Endo,Difference_In_Median,P_Value
A2ML1,1.26120,0.165194,-1.10585,0.037044,NaN,NaN
AADAT,1.33200,0.027477,0.18535,0.690613,NaN,NaN
AAGAB,0.09455,0.909622,-0.24350,0.139632,-0.1390,0.039189
AASDHPPT,-0.51445,0.003620,0.10590,0.428884,0.0628,0.973515
AATF,0.16905,0.042271,0.18170,0.915482,-0.0250,0.568821
...,...,...,...,...,...,...
ZNF654,-0.49290,0.048872,0.11860,0.294513,NaN,NaN
ZNF768,0.35120,0.069655,-0.19580,0.661851,-0.3755,0.030999
ZNHIT3,0.01085,0.990265,-0.13895,0.029161,NaN,NaN
ZNRD1,0.18620,0.423039,-0.30475,0.017492,NaN,NaN


In [35]:
significant.to_csv("csv_files/proteomics.csv")

In [24]:
# endo_and_colon = pd.concat([brca_df, endo_and_colon], axis=1)

In [25]:
# final_table = pd.concat([endo_and_colon, brca_df], axis=1, sort=False)

In [26]:
# final_table